# Создаём свой когнитивный портрет

Привет! Сейчас мы погрузимся в мир **Science Art**, где наука -- а именно **искусственный интеллект** -- поможет нам создавать произведения искусства! А именно, мы познакомимся с техникой [когнитивного портрета](http://bit.do/peopleblending).

<img src="https://soshnikov.com/images/art/PhoBoGuy.png" width="30%"/>

В этой технике мы используем **нейросеть** для выделения ключевых точек лица на серии фотографий, с последующим поворотом и масштабированием фотографий таким образом, чтобы глаза и рот на всех снимках совпадали. Совмещая такие снимки, мы получаем интересный визуальный эффект. 



## Загружаем фотографии

Для экспериментов нам потребуется несколько изображений. Поместите изображения в директорию `images`. В демонстрационных целях я буду использовать набор фотографий, который скачаю в следующей ячейке. Если же в хотите нарисовать свой портрет - то загрузите туда свои изобажения

In [ ]:
!wget http://www.soshnikov.com/permanent/data/dsh-images.zip
!unzip dsh-images.zip

Далее скачиваем и импортируем все необходимые библиотеки:

In [ ]:
!wget https://raw.githubusercontent.com/yandex-datasphere/jacademy/refs/heads/main/2025/jlib.py

In [ ]:
from jlib import *

Можем посмотреть, как выглядят наши фотографии:

In [ ]:
photos = [load_image(x) for x in sample_files("images/*.*")]
display_images(photos)

## Обнаруживаем лица на фотографии

Для начала, научимся находить лица на фотографии:

In [ ]:
img = load_image(sample_file("images/*.*"))
display_images(img)

Находим лица:

In [ ]:
faces = detect_faces(img)
print(f"Найдено {len(faces)} лиц")
plot_face(img,faces)

### Извлекаем опорные точки лица

Далее мы выбираем какое-то лицо на фотографии и извлекаем для него **опорные точки**.

In [ ]:
pts = detect_keypoints(img,faces[0])

plot_face(img,faces,pts)

### Обработка фотографий

Теперь мы пройдёмся по всем изображениям в нашей директории и извлечём из них опорные точки. Если на фотографии найдено несколько лиц - будем брать первое из них. Если лиц не найдено - будем эту фотографию игнорировать.

In [ ]:
from glob import glob

images = []
imagepoints = []

for fn in glob("images/*"):
    print(f"Обрабатываю картинку {fn}..",end='')
    try:
      img = load_image(fn)
      faces = detect_faces(img)
      print(f' найдено {len(faces)} лиц')
      if len(faces)>0:
          images.append(img)
          imagepoints.append(detect_keypoints(img,faces[0]))
    except:
      print('ошибка')

**Важно**: Если вдруг вы увидите сообщение **ОШИБКА**, это означает, фотография не смогла быть загружена, или возникла какая-то другая проблема. Такая фотография будет проигнорирована. Если несколько картинок оказались с ошибкой - не беда, главное, чтобы какое-то количество лиц было найдено!

Посмотрим на несколько загруженных нами изображений:

In [ ]:
display_images(images[:5])

## Преобразования изображений

Чтобы выровнять все изображения по глазам, нам необходимо повернуть и масштабировать их. Для этого используем функцию `transform_image`, куда передаём координаты опорных точек, размер целевого изображения, и координаты глаз и середины рта на изображении.

Будем ориентироваться на размер финального изображения $300\times300$ точек, тогда для координат глаз выберем точки $(130,120)$ и $(170,120)$. 

In [ ]:
size = 600

target_triangle = [
    [130.0/300*size,120.0/300*size],
    [170.0/300*size,120.0/300*size],
    [150.0/300*size,160.0/300*size]]

img_aligned = [transform_image(i,a,size,target_triangle) for i,a in zip(images,imagepoints)]
display_images(img_aligned[:5])

## Смешивание изображений

Для получения результата нам осталось смешать все эти изображения:

In [ ]:
res = merge(img_aligned)
display_images(res)

Получившийся результат можно записать:

In [ ]:
save_image(res,'result.jpg')

## Выводы

Python позволяет легко работать с изображениями и с нейросетями. Это позволяет нам решать сложные задачи несколькими строчками кода!